In [ ]:
#importing required libraries

import itertools
import cv2
import os
import time

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.applications.efficientnet import EfficientNetB7  as PretrainedModel, preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from glob import glob

In [ ]:
# Data preparation
folders = glob('D:/tester-esophagous' + '/*')
print('New Paths: ', folders)

IMAGE_FILES = glob('D:/tester-esophagous/endoscopy' + '/*/*.*')
print('Images Count: ', len(IMAGE_FILES))

SAMPLES = [
    "D:/tester-esophagous/endoscopy/noesophagous/0b792e26-e1dd-4fb7-a6b7-5d76f227a677.jpg",
    "D:/tester-esophagous/endoscopy/esophagous/0c61343a-52a3-4f9b-85c3-732637968885.jpg",    
    "D:/tester-esophagous/endoscopy/noesophagous/0c4091c0-2f80-4473-8362-041200f056ba.jpg",
    "D:/tester-esophagous/endoscopy/esophagous/0f8d0452-78f0-4f1b-922c-ffdd56a57a4a.jpg"
]

plt.figure(figsize=(20, 10))
for i, img_path in enumerate(SAMPLES):
    img = image.load_img(img_path, target_size=(224, 224))
    plt.subplot(1, len(SAMPLES), i+1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [ ]:
# Model preparation
base_model = PretrainedModel(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
EsophagealCancer_Model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

EsophagealCancer_Model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Data generators
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    'D:/tester-esophagous/endoscopy',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'D:/tester-esophagous/endoscopy',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
# Training the model
history = EsophagealCancer_Model.fit(
    train_generator, 
    validation_data=validation_generator, 
    epochs=3, 
    verbose=1, 
)

In [ ]:
# Plotting accuracy and loss
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('Training & Validation Accuracy', fontsize=16, weight='bold')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.show()

In [ ]:
# Saving the model
EsophagealCancer_Model.save('D:/tester-esophagous/model/EsophagealCancerModel.keras')

In [ ]:
#Shows output with help of background code
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

#Code to predict the Esophageal Cancer based on the accuracy/confidence
def predict_esophageal_cancer(image_path, model_path='D:/tester-esophagous/model/EsophagealCancerModel.keras'):
    # Load the trained model
    model = load_model(model_path)
   
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
   
    # Make prediction
    prediction = model.predict(img_array)
   
    # Interpret the result
    if prediction[0][0] > 0.5:
        result = "Esophageal cancer detected"
        probability = prediction[0][0]
    else:
        result = "No esophageal cancer detected"
        probability = 1 - prediction[0][0]
   
    return result, probability


In [ ]:
# Example usage
image_path = 'D:/tester-esophagous/Esophagous-cancer-detection-testing-image/images.jpg'  # Replace with the path to your image
result, probability = predict_esophageal_cancer(image_path)
print(f"Prediction: {result}")
print(f"Confidence: {probability:.2%}")